In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [15]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-mini'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [16]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [17]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [18]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [19]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [20]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic) **** Strongly typed variables
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [21]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [22]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [23]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [24]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2025-10-23 17:57:52,249 - 140420945946432 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [26]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: To find the perfect venue in San Francisco for the Tech Innovation Conference, I need to understand the specific event requirements such as expected attendance size, budget constraints, preferred venue type (conference center, hotel, auditorium, etc.), and any special facilities needed (AV equipment, breakout rooms, catering, accessibility, etc.). I should first clarify these details to narrow down the search effectively.

Action: Ask question to co-worker  
Action Input: {"coworker": "Logistics Manager", "question": "Could you please provide the event requirements for the Tech Innovation Conference, including expected attendance, budget, preferred venue type, and any special needs such as AV equipment, breakout rooms, or catering?"}  

Thought: Awaiting detailed event requirements

Thought: I need to first confirm the venue booking status and capacity details to ensure the Moscone Center can accommodate at least 500 attendees or if alternative arrangements are needed. Then, I will outline a comprehensive marketing plan targeting 500+ attendees for the Tech Innovation Conference.

Action: Ask question to co-worker  
Action Input: {"coworker": "Venue Coordinator", "question": "Please confirm the booking status of the Moscone Center located at 747 Howard Street, San Francisco, CA 94103 for the Tech Innovation Conference. Also, can the venue accommodate at least 500 attendees or will we need to consider alternative venues or arrangements?"}  
 

I encountered an error while trying to use the tool. This was the error: AgentTools.ask_question() missing 1 required positional argument: 'context'.
 Tool Ask question to co-worker accepts these inputs: Ask question to co-worker(coworker: str, question: str, context: str) - Ask a specific question to one of the following co-

In [27]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '747 Howard Street, San Francisco, CA 94103',
 'booking_status': 'To be confirmed',
 'capacity': 300,
 'name': 'Moscone Center'}


In [28]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md